In [1]:
%load_ext line_profiler
%load_ext memory_profiler



In [4]:
tokenizer = AutoTokenizer.from_pretrained('airesearchth/wangchanberta-base-wiki-20210520-spm')

In [6]:
SPECIAL_TOKEN_NAMES = ['bos_token', 'eos_token', 'sep_token', 'pad_token', 'cls_token', 'mask_token']


In [288]:
pvals = 1. / np.arange(1, 3 + 1)
print('pvals', pvals)
pvals /= pvals.sum(keepdims=True)
print('pvals norm', pvals)

print('pvals sum', sum(pvals))

pvals [1.         0.5        0.33333333]
pvals norm [0.54545455 0.27272727 0.18181818]
pvals sum 1.0000000000000002


In [309]:
# p=0.2
# k=3

# [ geometric_dist(k=k, p=p)for k in range(1,3+1)]

In [290]:
tokenizer, tokenizer.mask_token

(PreTrainedTokenizer(name_or_path='airesearchth/wangchanberta-base-wiki-20210520-spm', vocab_size=24005, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '▁']}),
 '<mask>')

In [318]:
vals = torch.LongTensor([tokenizer.convert_tokens_to_ids(['<s>', '</s>', '<unk>', '</s>', '<pad>', '<s>', '<mask>', '<s>NOTUSED', '</s>NOTUSED', '▁', 'ก'])])
print(vals)

[ tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in vals]

tensor([[    5,     6,     3,     6,     1,     5, 24004,     0,     2,     8,
            58]])


[[1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]]

In [292]:
tokenizer.special_tokens_map
# '<s>', '</s>', '<unk>', '</s>', '<pad>', '<s>', '<mask>', '<s>NOTUSED', '</s>NOTUSED', '▁'

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>',
 'additional_special_tokens': "['<s>NOTUSED', '</s>NOTUSED', '▁']"}

In [302]:
mask_generator = NGramMaskGenerator(tokenizer, max_gram=3, mlm_prob=0.15)

 self.max_preds_per_seq 80


In [303]:
# text = "We present SpanBERT, a pre-training method that is designed to better represent and predict spans of text."
text = """ภาษาอินโด-ยูโรเปียนดั้งเดิม คือภาษาสืบสร้างซึ่งเป็นภาษาจากการสมมุติฐานที่คาดว่าบรรพบุรุษของอินโด-ยูโรเปียนเคยพูดภาษาเดียวกัน ภาษาอินโด-ยูโรเปียนดั้งเดิมคือภาษาดั้งเดิมที่เป็นที่สนใจมากที่สุด 
รวมถึงเข้าใจมากที่สุดอีกด้วย งานส่วนใหญ่ของนักภาษาศาสตร์ในช่วงศตวรรษที่ 19 มักจะเป็นเรื่องบูรณะภาษานี้ และภาษาลูกหลานเช่น ภาษาเจอร์แมนิกดั้งเดิม"""
tokens = tokenizer.tokenize(text)
print(tokens, len(tokens))


['ภาษา', 'อินโด', '-', 'ยูโรเปียน', 'ดั้งเดิม', '▁', 'คือ', 'ภาษา', 'สืบ', 'สร้าง', 'ซึ่งเป็น', 'ภาษา', 'จากการ', 'สมมุติฐาน', 'ที่', 'คาดว่า', 'บรรพบุรุษ', 'ของ', 'อินโด', '-', 'ยูโรเปียน', 'เคย', 'พูดภาษา', 'เดียวกัน', '▁', 'ภาษา', 'อินโด', '-', 'ยูโรเปียน', 'ดั้งเดิม', 'คือ', 'ภาษา', 'ดั้งเดิม', 'ที่เป็น', 'ที่สนใจ', 'มากที่สุด', '▁', 'รวมถึง', 'เข้าใจ', 'มากที่สุด', 'อีกด้วย', '▁', 'งาน', 'ส่วนใหญ่', 'ของนัก', 'ภาษาศาสตร์', 'ในช่วง', 'ศตวรรษที่', '▁', '19', '▁', 'มักจะเป็น', 'เรื่อง', 'บูรณะ', 'ภาษานี้', '▁', 'และภาษา', 'ลูกหลาน', 'เช่น', '▁', 'ภาษา', 'เจอร์แมนิก', 'ดั้งเดิม'] 63


In [304]:
rng = random


In [305]:
tokens, labels = mask_generator.mask_tokens(tokens=tokens, rng=rng)
print(tokens)

['ภาษา', 'อินโด', '-', 'ยูโรเปียน', 'ดั้งเดิม', '<mask>', 'คือ', 'ภาษา', '<mask>', 'สร้าง', 'ซึ่งเป็น', 'ภาษา', '<mask>', 'สมมุติฐาน', 'ที่', 'คาดว่า', 'บรรพบุรุษ', 'ของ', 'อินโด', '<mask>', 'ยูโรเปียน', 'เคย', 'พูดภาษา', 'เดียวกัน', '▁', 'ภาษา', 'อินโด', '<mask>', 'ยูโรเปียน', 'ดั้งเดิม', 'คือ', 'ภาษา', 'ดั้งเดิม', '<mask>', 'ที่สนใจ', 'มากที่สุด', '▁', 'รวมถึง', 'เข้าใจ', 'มากที่สุด', '<mask>', '▁', 'งาน', 'ส่วนใหญ่', '<mask>', 'ภาษาศาสตร์', 'ในช่วง', 'ศตวรรษที่', '▁', '19', '▁', '<mask>', 'เรื่อง', 'บูรณะ', 'ภาษานี้', '▁', 'และภาษา', 'ลูกหลาน', 'เช่น', '▁', 'ภาษา', 'เจอร์แมนิก', 'ดั้งเดิม']


In [306]:
print(tokenizer.convert_ids_to_tokens(labels))

['<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '▁', '<s>NOTUSED', '<s>NOTUSED', 'สืบ', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', 'จากการ', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '-', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '-', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', 'ที่เป็น', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', 'อีกด้วย', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', 'ของนัก', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', 'มักจะเป็น', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED', '<s>NOTUSED']
